In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce
from tabulate import tabulate

In [9]:
total = []
std = pd.read_csv('../Data/Intermediate/COVIDcounty_cases_processed_2020.csv', index_col = 0)
dp05 = pd.read_csv('../Data/Intermediate/ACSDP05_merge.csv', index_col = 0)
dp03 = pd.read_csv('../Data/Intermediate/ACSDP03_merge.csv', index_col = 0)
saipe = pd.read_csv('../Data/Intermediate/saipe0010.csv', index_col = 0)
hps = pd.read_csv('../Data/Intermediate/hps_processed.csv', index_col = 0)
total.extend([std, dp05, dp03, saipe])
#print(total)

In [10]:
for index, i in enumerate(total):
    i['fips'] = i['fips'].astype(str).str.zfill(5)
    #print(list(i.columns))
    if 'year' in i.columns:
        if 'area' in i.columns:
            i.drop(columns = 'area', inplace = True)
            print(index, ': ', 'Done')
        if 'county_name' in i.columns:
            i.drop(columns = 'county_name' , inplace = True)
            print(index, ': ', 'done')
        if 'state_fips' in i.columns:
            i.drop(columns = 'state_fips', inplace = True)
            print(index, ': ', 'DONE')

0 :  done


In [4]:
cov = total[0].columns.intersection(total[1].columns)
df = pd.merge(total[0], total[1], on = cov.to_list(), how = 'outer')
df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
print(df['year'].value_counts())
print(df.shape)
df.head(20)

2020    3222
2010    3221
2011    3221
2012    3221
2013    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
(35426, 43)


,year,fips,covid_cases,pct_one_or_more_race,pct_one_or_more_race_white,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,...,pct_sex_age_65+_female,total_pop,pct_sex_age_male,pct_sex_age_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,pct_other_race
0,2010,01001,NaN,53155.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,53155.0,48.5,51.5,80.4,18.2,1.0,1.1,0.1,0.7
1,2011,01001,NaN,53944.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,53944.0,48.5,51.5,80.4,18.4,1.0,1.2,0.0,0.7
2,2012,01001,NaN,54590.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,54590.0,48.6,51.4,80.3,18.5,1.0,1.2,0.0,0.5
3,2013,01001,NaN,54907.0,79.6,18.8,0.9,1.3,0.0,0.8,...,57.2,54907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,01001,NaN,55136.0,79.2,19.1,0.9,1.3,0.0,0.8,...,57.7,55136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,01001,NaN,55221.0,78.8,19.2,0.9,1.4,0.3,1.1,...,57.0,55221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016,01001,NaN,55049.0,78.7,19.3,0.9,1.4,0.2,1.5,...,56.7,55049.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,01001,NaN,55036.0,78.2,19.6,0.8,1.5,0.2,1.3,...,56.5,55036.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,01001,NaN,55200.0,78.6,19.9,0.8,1.6,0.2,0.8,...,56.7,55200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019,01001,NaN,55380.0,78.8,20.0,0.8,1.7,0.2,0.7,...,56.6,55380.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for i in total[2:9]:
    cov = df.columns.intersection(i.columns)
    df = pd.merge(df, i, on = cov.to_list(), how = 'outer')
    df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
    df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
    print(df['year'].value_counts())
    print(df.shape)

2020    3222
2010    3221
2011    3221
2012    3221
2013    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: year, dtype: int64
(35426, 72)
2020    3222
2010    3221
2012    3221
2013    3221
2011    3221
2017    3220
2018    3220
2019    3220
2016    3220
2015    3220
2014    3220
2009    3143
2008    3142
2002    3141
2003    3141
2004    3141
2005    3141
2006    3141
2007    3141
Name: year, dtype: int64
(60557, 80)


In [6]:
print(df.dtypes)
df.head(30)

year                            int64
fips                           object
covid_cases                   float64
pct_one_or_more_race          float64
pct_one_or_more_race_white    float64
                               ...   
per_capita_income_2008        float64
per_capita_income_2003        float64
per_capita_income_2004        float64
per_capita_income_2009        float64
per_capita_income_2002        float64
Length: 80, dtype: object


,year,fips,covid_cases,pct_one_or_more_race,pct_one_or_more_race_white,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,...,per_capita_income_2018,per_capita_income_2013,per_capita_income_2007,per_capita_income_2006,per_capita_income_2005,per_capita_income_2008,per_capita_income_2003,per_capita_income_2004,per_capita_income_2009,per_capita_income_2002
0,2010,01001,NaN,53155.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,01001,NaN,53944.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,01001,NaN,54590.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,01001,NaN,54907.0,79.6,18.8,0.9,1.3,0.0,0.8,...,NaN,24571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,01001,NaN,55136.0,79.2,19.1,0.9,1.3,0.0,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,01001,NaN,55221.0,78.8,19.2,0.9,1.4,0.3,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016,01001,NaN,55049.0,78.7,19.3,0.9,1.4,0.2,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,01001,NaN,55036.0,78.2,19.6,0.8,1.5,0.2,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,01001,NaN,55200.0,78.6,19.9,0.8,1.6,0.2,0.8,...,29372.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019,01001,NaN,55380.0,78.8,20.0,0.8,1.7,0.2,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df[(df['fips']=='01001')]

,year,fips,covid_cases,pct_one_or_more_race,pct_one_or_more_race_white,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,...,per_capita_income_2018,per_capita_income_2013,per_capita_income_2007,per_capita_income_2006,per_capita_income_2005,per_capita_income_2008,per_capita_income_2003,per_capita_income_2004,per_capita_income_2009,per_capita_income_2002
0,2010,01001,NaN,53155.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,01001,NaN,53944.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,01001,NaN,54590.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,01001,NaN,54907.0,79.6,18.8,0.9,1.3,0.0,0.8,...,NaN,24571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,01001,NaN,55136.0,79.2,19.1,0.9,1.3,0.0,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,01001,NaN,55221.0,78.8,19.2,0.9,1.4,0.3,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016,01001,NaN,55049.0,78.7,19.3,0.9,1.4,0.2,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,01001,NaN,55036.0,78.2,19.6,0.8,1.5,0.2,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,01001,NaN,55200.0,78.6,19.9,0.8,1.6,0.2,0.8,...,29372.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019,01001,NaN,55380.0,78.8,20.0,0.8,1.7,0.2,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv('../Data/Final/STI0220_cov0220.csv')